# Erica AI Tutor - Environment Verification

This notebook verifies that all services are running correctly.

**Your Setup:**
- MacBook M1 Pro
- OpenRouter API for LLM (Qwen2.5)
- Docker containers for databases

Run each cell to check connectivity.

In [1]:
# Add src to path
import sys
sys.path.insert(0, '/app/src')

from config import settings
print("Configuration loaded")
print(f"  OpenRouter Model: {settings.openrouter_model}")
print(f"  API Key Set: {'Yes' if settings.openrouter_api_key else 'NO - Please set OPENROUTER_API_KEY in .env'}")
print(f"  Neo4j URI: {settings.neo4j_uri}")
print(f"  ChromaDB: {settings.chroma_url}")

Configuration loaded
  OpenRouter Model: qwen/qwen-2.5-72b-instruct
  API Key Set: Yes
  Neo4j URI: bolt://neo4j:7687
  ChromaDB: http://chromadb:8000


## 1. Test OpenRouter API Connection

In [2]:
from llm import LLMClient, AVAILABLE_MODELS

def test_openrouter():
    if not settings.openrouter_api_key:
        print("FAIL: OpenRouter API key not set")
        print("  1. Get your key at: https://openrouter.ai/keys")
        print("  2. Add to .env: OPENROUTER_API_KEY=sk-or-v1-your-key")
        print("  3. Restart the container: docker-compose restart app")
        return False
    
    try:
        client = LLMClient()
        response = client.generate(
            "What is 2+2? Reply with just the number.",
            temperature=0
        )
        print("OK: OpenRouter API working")
        print(f"  Model: {settings.openrouter_model}")
        print(f"  Test response: {response.strip()}")
        return True
    except Exception as e:
        print(f"FAIL: OpenRouter API error: {e}")
        return False

print("Available Qwen models on OpenRouter:")
for name, model_id in AVAILABLE_MODELS.items():
    print(f"  {name}: {model_id}")
print()

test_openrouter()

Available Qwen models on OpenRouter:
  qwen2.5-72b: qwen/qwen-2.5-72b-instruct
  qwen2.5-32b: qwen/qwen-2.5-32b-instruct
  qwen2.5-14b: qwen/qwen-2.5-14b-instruct
  qwen2.5-7b: qwen/qwen-2.5-7b-instruct
  qwen2.5-coder-32b: qwen/qwen-2.5-coder-32b-instruct

OK: OpenRouter API working
  Model: qwen/qwen-2.5-72b-instruct
  Test response: 4


True

## 2. Test Neo4j Connection

In [3]:
from neo4j import GraphDatabase

def test_neo4j():
    try:
        driver = GraphDatabase.driver(
            settings.neo4j_uri,
            auth=(settings.neo4j_user, settings.neo4j_password)
        )
        with driver.session() as session:
            result = session.run("RETURN 1 as n")
            result.single()
        driver.close()
        print("OK: Neo4j is running")
        print(f"  Browser UI: http://localhost:7474")
        print(f"  Login: neo4j / erica_password_123")
        return True
    except Exception as e:
        print(f"FAIL: Cannot connect to Neo4j: {e}")
        print("  Make sure the neo4j container is running: docker-compose up -d neo4j")
    return False

test_neo4j()

OK: Neo4j is running
  Browser UI: http://localhost:7474
  Login: neo4j / erica_password_123


True

## 3. Test ChromaDB Connection

In [6]:
import chromadb

def test_chromadb():
    try:
        client = chromadb.HttpClient(
            host=settings.chroma_host,
            port=settings.chroma_port
        )
        # Try to list collections
        collections = client.list_collections()
        print("OK: ChromaDB is running")
        print(f"  Collections: {len(collections)}")
        return True
    except Exception as e:
        print(f"FAIL: Cannot connect to ChromaDB: {e}")
        print("  Make sure the chromadb container is running: docker-compose up -d chromadb")
    return False

test_chromadb()

OK: ChromaDB is running
  Collections: 0


True

## 4. Test MongoDB Connection

In [7]:
from pymongo import MongoClient

def test_mongodb():
    try:
        client = MongoClient(settings.mongodb_uri, serverSelectionTimeoutMS=5000)
        # Force connection
        client.server_info()
        db = client[settings.mongodb_database]
        print("OK: MongoDB is running")
        print(f"  Database: {settings.mongodb_database}")
        print(f"  Web UI: http://localhost:8081")
        client.close()
        return True
    except Exception as e:
        print(f"FAIL: Cannot connect to MongoDB: {e}")
        print("  Make sure the mongodb container is running: docker-compose up -d mongodb")
    return False

test_mongodb()

OK: MongoDB is running
  Database: erica_tutor
  Web UI: http://localhost:8081


True

## 5. Test Embedding Model

In [8]:
from sentence_transformers import SentenceTransformer

def test_embeddings():
    try:
        print(f"Loading embedding model: {settings.embedding_model}...")
        model = SentenceTransformer(settings.embedding_model)
        test_text = "Introduction to Artificial Intelligence"
        embedding = model.encode(test_text)
        print("OK: Embedding model loaded")
        print(f"  Model: {settings.embedding_model}")
        print(f"  Embedding dimension: {len(embedding)}")
        return True
    except Exception as e:
        print(f"FAIL: Cannot load embedding model: {e}")
    return False

test_embeddings()

Loading embedding model: all-MiniLM-L6-v2...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

OK: Embedding model loaded
  Model: all-MiniLM-L6-v2
  Embedding dimension: 384


True

## 6. Test Entity Extraction (LLM + Structured Output)

In [9]:
def test_entity_extraction():
    """Test that we can extract entities - crucial for M3 (Knowledge Graph)."""
    if not settings.openrouter_api_key:
        print("SKIP: OpenRouter not configured")
        return False
    
    try:
        from llm import LLMClient
        client = LLMClient()
        
        sample_text = """
        Machine learning is a subset of artificial intelligence that enables 
        systems to learn from data. Supervised learning requires labeled data, 
        while unsupervised learning finds patterns without labels. 
        Neural networks are a key technique in deep learning.
        """
        
        result = client.extract_entities(sample_text)
        
        print("OK: Entity extraction working")
        print(f"  Entities found: {len(result.get('entities', []))}")
        print(f"  Relationships found: {len(result.get('relationships', []))}")
        
        if result.get('entities'):
            print("\n  Sample entities:")
            for ent in result['entities'][:3]:
                print(f"    - {ent.get('name', 'N/A')} ({ent.get('type', 'N/A')})")
        
        return True
    except Exception as e:
        print(f"FAIL: Entity extraction failed: {e}")
        return False

print("Testing entity extraction (this calls the LLM)...")
test_entity_extraction()

Testing entity extraction (this calls the LLM)...
OK: Entity extraction working
  Entities found: 6
  Relationships found: 5

  Sample entities:
    - Machine learning (concept)
    - Supervised learning (concept)
    - Unsupervised learning (concept)


True

## Summary

In [10]:
print("\n" + "="*50)
print("ENVIRONMENT VERIFICATION SUMMARY")
print("="*50 + "\n")

results = {
    "OpenRouter (LLM)": test_openrouter(),
    "Neo4j (Graph DB)": test_neo4j(),
    "ChromaDB (Vector DB)": test_chromadb(),
    "MongoDB (Document Store)": test_mongodb(),
    "Embeddings": test_embeddings()
}

print("\n" + "-"*50)
all_pass = all(results.values())
if all_pass:
    print("\nSUCCESS: All services are running! You are ready for M2 (Ingestion).")
    print("\nNext steps:")
    print("  1. Open notebooks/01_ingestion.ipynb")
    print("  2. Start ingesting course content")
else:
    failed = [k for k, v in results.items() if not v]
    print(f"\nWARNING: Some services failed: {failed}")
    print("  Fix these before proceeding.")


ENVIRONMENT VERIFICATION SUMMARY

OK: OpenRouter API working
  Model: qwen/qwen-2.5-72b-instruct
  Test response: 4
OK: Neo4j is running
  Browser UI: http://localhost:7474
  Login: neo4j / erica_password_123
OK: ChromaDB is running
  Collections: 0
OK: MongoDB is running
  Database: erica_tutor
  Web UI: http://localhost:8081
Loading embedding model: all-MiniLM-L6-v2...
OK: Embedding model loaded
  Model: all-MiniLM-L6-v2
  Embedding dimension: 384

--------------------------------------------------

SUCCESS: All services are running! You are ready for M2 (Ingestion).

Next steps:
  1. Open notebooks/01_ingestion.ipynb
  2. Start ingesting course content


---

## Troubleshooting

### OpenRouter Issues
```bash
# Check your API key is set
grep OPENROUTER .env

# Restart container after changing .env
docker-compose restart app
```

### Container Issues (Mac M1)
```bash
# Check container status
docker-compose ps

# View logs for a specific service
docker-compose logs neo4j
docker-compose logs chromadb

# Restart all containers
docker-compose down && docker-compose up -d
```

### Memory Issues on Mac
- Increase Docker Desktop memory allocation (Settings > Resources)
- Recommended: 8GB+ for all services